In [0]:
With L1 as (select year(TradingDate) as `Year`, month(tradingdate) as `Month`,
ConstraintTypeCode, ConstraintDirectionIndicator, sum(Dollars) as Total_Dollars
from ea_prd.gold.factconstrainedonoff 
where TRadingDate >= '2024-01-01'
group by all
), L2 as (select year(TradingDate) as `Year`, month(tradingdate) as `Month`,
ConstraintTypeCode, ConstraintDirectionIndicator, sum(Dollars) as Rotohiko_Dollars
from ea_prd.gold.factconstrainedonoff 
where TRadingDate >= '2024-01-01' and PointOfConnectionCode = 'HLY0331' and ParticipantCode = 'NPEL'
group by all
)
Select L1.* , L2. Rotohiko_Dollars
from L1 left join L2 
on L1.Year = L2.Year and L1.Month = L2.Month 
and L1.ConstraintTypeCode = L2.ConstraintTypeCode 
and L1.ConstraintDirectionIndicator = L2.ConstraintDirectionIndicator
where (l1.constrainttypecode <> 'FKCONS' )
--and not (l1.constrainttypecode = 'ECONS' and l1.constraintdirectionindicator = 'OFF')
order by all

In [0]:
select PNodeCode, ParticipantCode, ConstraintTypeCode, ConstraintDirectionIndicator, 
Min(TradingDate) as FromDate, Max(TradingDate) as ToDate, sum(Dollars) as Dollars, avg(Megawatts) as Average_MW
from ea_prd.gold.factconstrainedonoff 
where PointOfConnectionCode in ('HLY0331','BRB0331','SWN2201')
group by all

In [0]:
With L1 as (
  select TradingDate, TradingPeriodNumber, PointOfConnectionCode,TraderCode,NetworkParticipantCode, KilowattHours/1000 as MWh from ea_prd.gold.factreconciledinjectionandofftake
  where TradingDate >= '2024-01-01' and FlowDirection = 'Injection' and KilowattHours > 0
  and (
      -- (PointOfConnectionCode = 'BRB0331' and TraderCode in ('MERI') and NetworkParticipantCode = 'MERI')
    -- or 
      (PointOfConnectionCode = 'HLY0331' and TraderCode in ('NPEL','OPHL') and NetworkParticipantCode = 'WAIK')
      )
), L2 as (
  select distinct TradingDate, TradingPeriodNumber, PointOfConnectionCode, DollarsPerMegawattHour from ea_prd.gold.factpredispatchandfinalenergyprices
  where TradingDate >= '2024-01-01' and PointOfConnectionCode in ('BRB0331','HLY0331') and PriceTypeCode = 'F'
)
select Year(L1.TradingDate) as `Year`, Month(L1.TradingDate) as `Month`,
round(sum(MWh),3) as Total_injection_MWh, round(sum(DollarsPerMegawattHour*MWh),2) as Generation_revenue
from L1 inner join L2 
on L1.TradingDate = L2.TradingDate 
and L1.TradingPeriodNumber = L2.TradingPeriodNumber 
and L1.PointOfConnectionCode = L2.PointOfConnectionCode
group by all
Order by Year, Month

In [0]:
With L1 as (
  select TradingDate, TradingPeriodNumber, PointOfConnectionCode,TraderCode,NetworkParticipantCode, KilowattHours/1000 as MWh from ea_prd.gold.factreconciledinjectionandofftake
  where TradingDate >= '2024-01-01' and FlowDirection = 'Offtake' and KilowattHours > 0
  and (
      -- (PointOfConnectionCode = 'BRB0331' and TraderCode in ('MERI') and NetworkParticipantCode = 'MERI')
    -- or 
      (PointOfConnectionCode = 'HLY0331' and TraderCode in ('NPEL','OPHL') and NetworkParticipantCode = 'WAIK')
      )
), L2 as (
  select distinct TradingDate, TradingPeriodNumber, PointOfConnectionCode, DollarsPerMegawattHour from ea_prd.gold.factpredispatchandfinalenergyprices
  where TradingDate >= '2024-01-01' and PointOfConnectionCode in ('BRB0331','HLY0331') and PriceTypeCode = 'F'
)
select Year(L1.TradingDate) as `Year`, Month(L1.TradingDate) as `Month`,
round(sum(MWh),3) as Total_injection_MWh, round(sum(DollarsPerMegawattHour*MWh),2) as Load_cost
from L1 inner join L2 
on L1.TradingDate = L2.TradingDate 
and L1.TradingPeriodNumber = L2.TradingPeriodNumber 
and L1.PointOfConnectionCode = L2.PointOfConnectionCode
group by all
Order by Year, Month

In [0]:
with L1 as (
  select DATE(TRadingDate) as TradingDate, int(TradingPeriod) as TradingPeriodNumber, CaseID, int(InterimTimeWeightSeconds) as NumberofSecs 
  from ea_prd.bronze.casefilepublishtimes where InterimTimeWeightSeconds <> '0' 
), L2 as (
  Select a.CaseID, a.TradingDate, a.TradingPeriodNumber, a.IntervalDateTime, b.NumberofSecs,
  avg(a.SPDClearedFIRMegawatts) as Cleared_FIR_MW, avg(a.SPDClearedFIRMegawatts) as Cleared_SIR_MW
  from ea_prd.gold.factspdbidsandoffers a inner join L1 b
  on (a.TradingDate = b.TradingDate and a.TradingPeriodNumber = b.TradingPeriodNumber and a.CaseID = b.CaseID)
  where a.TradingDate >= '2024-01-01' 
  and a.PointOfConnectionCode = 'HLY0331'
  --and UnitCode in ('RHO0','RHO99')
  and OfferTypeCode not in ('ENOF','ENDL')
  Group By all
), L3 as (
  select TradingDate, TradingPeriodNumber, 
  round(sum(Cleared_FIR_MW*NumberofSecs)/sum(NumberofSecs),3) as Cleared_FIR_MW, 
  round(sum(Cleared_SIR_MW*NumberofSecs)/sum(NumberofSecs),3) as Cleared_SIR_MW
  from L2
  Group by TradingDate, TradingPeriodNumber
), L4 as (
select a.*, b.FIRDollarsPerMegawattHour, b.SIRDollarsPerMegawattHour 
from L3 a inner join ea_prd.gold.factpredispatchandfinalreserveprices b
on (a.TradingDate = b.TradingDate and a.TradingPeriodNumber = b.TradingPeriodNumber 
and b.PointOfConnectionCode = 'HAY2201' and b.PriceTypeCode = 'F')
)
select Year(TradingDate) as `Year`, month(TradingDate) as `Month`, 
round(sum(Cleared_FIR_MW * FIRDollarsPerMegawattHour),2) as Revenue_FIR, 
round(sum(Cleared_SIR_MW * SIRDollarsPerMegawattHour),2) as Revenue_SIR 
from L4
Group by all
Order by Year, Month

#TEST CODE

In [0]:
with L1 as (
  select DATE(TRadingDate) as TradingDate, int(TradingPeriod) as TradingPeriodNumber, CaseID, int(InterimTimeWeightSeconds) as NumberofSecs 
  from ea_prd.bronze.casefilepublishtimes where InterimTimeWeightSeconds <> '0' 
), L2 as (
  Select a.CaseID, a.TradingDate, a.TradingPeriodNumber, a.IntervalDateTime, b.NumberofSecs,
  avg(a.SPDClearedFIRMegawatts) as Cleared_FIR_MW, avg(a.SPDClearedFIRMegawatts) as Cleared_SIR_MW
  from ea_prd.gold.factspdbidsandoffers a inner join L1 b
  on (a.TradingDate = b.TradingDate and a.TradingPeriodNumber = b.TradingPeriodNumber and a.CaseID = b.CaseID)
  where a.TradingDate >= '2024-01-01' 
  and a.PointOfConnectionCode = 'HLY0331'
  --and UnitCode in ('RHO0','RHO99')
  and OfferTypeCode not in ('ENOF','ENDL')
  Group By all
), L3 as (
  select TradingDate, TradingPeriodNumber, 
  round(sum(Cleared_FIR_MW*NumberofSecs)/sum(NumberofSecs),3) as Cleared_FIR_MW, 
  round(sum(Cleared_SIR_MW*NumberofSecs)/sum(NumberofSecs),3) as Cleared_SIR_MW
  from L2
  Group by TradingDate, TradingPeriodNumber
), L4 as (
select a.*, b.FIRDollarsPerMegawattHour, b.SIRDollarsPerMegawattHour 
from L3 a inner join ea_prd.gold.factpredispatchandfinalreserveprices b
on (a.TradingDate = b.TradingDate and a.TradingPeriodNumber = b.TradingPeriodNumber 
and b.PointOfConnectionCode = 'HAY2201' and b.PriceTypeCode = 'F')
)
select Year(TradingDate) as `Year`, month(TradingDate) as `Month`, 
round(sum(Cleared_FIR_MW)/2,2) as FIR_MWh, round(sum(Cleared_FIR_MW * FIRDollarsPerMegawattHour)/2,2) as Revenue_FIR, 
round(sum(Cleared_FIR_MW * FIRDollarsPerMegawattHour)/sum(Cleared_FIR_MW),2) as avg_FIR_Price,
round(sum(Cleared_SIR_MW)/2,2) as SIR_MWh, round(sum(Cleared_SIR_MW * SIRDollarsPerMegawattHour)/2,2) as Revenue_SIR,
round(sum(Cleared_SIR_MW * SIRDollarsPerMegawattHour)/sum(Cleared_SIR_MW),2) as avg_SIR_Price
from L4
Group by all
Order by Year, Month

In [0]:
With L1 as (
  select TradingDate, TradingPeriodNumber, PointOfConnectionCode,TraderCode,NetworkParticipantCode, KilowattHours/1000 as MWh from ea_prd.gold.factreconciledinjectionandofftake
  where TradingDate >= '2024-01-01' and FlowDirection = 'Injection' and KilowattHours > 0
  and ((PointOfConnectionCode = 'BRB0331' and TraderCode in ('MERI') and NetworkParticipantCode = 'MERI')
    or (PointOfConnectionCode = 'HLY0331' and TraderCode in ('NPEL','OPHL') and NetworkParticipantCode = 'WAIK')
  )
), L2 as (
  select distinct TradingDate, TradingPeriodNumber, PointOfConnectionCode, DollarsPerMegawattHour from ea_prd.gold.factpredispatchandfinalenergyprices
  where TradingDate >= '2024-01-01' and PointOfConnectionCode in ('BRB0331','HLY0331') and PriceTypeCode = 'F'
)
select L1.PointOfConnectionCode,TraderCode,NetworkParticipantCode, 
Min(L1.TradingDate) as FromDate, Max(L1.TradingDate) as ToDate,
round(sum(MWh),3) as Total_injection_MWh, round(sum(DollarsPerMegawattHour*MWh),2) as Generation_revenue
from L1 inner join L2 
on L1.TradingDate = L2.TradingDate 
and L1.TradingPeriodNumber = L2.TradingPeriodNumber 
and L1.PointOfConnectionCode = L2.PointOfConnectionCode
group by all
Order by PointOfConnectionCode

In [0]:
With L1 as (
  select TradingDate, TradingPeriodNumber, PointOfConnectionCode,TraderCode,NetworkParticipantCode, KilowattHours/1000 as MWh from ea_prd.gold.factreconciledinjectionandofftake
  where TradingDate >= '2024-01-01' and FlowDirection = 'Offtake' and KilowattHours > 0
  and ((PointOfConnectionCode = 'BRB0331' and TraderCode = 'MERI' and NetworkParticipantCode = 'MERI')
    or (PointOfConnectionCode = 'HLY0331' and TraderCode in ('NPEL','OPHL') --and NetworkParticipantCode = 'WAIK'
    )
  )
), L2 as (
  select distinct TradingDate, TradingPeriodNumber, PointOfConnectionCode, DollarsPerMegawattHour from ea_prd.gold.factpredispatchandfinalenergyprices
  where TradingDate >= '2024-01-01' and PointOfConnectionCode in ('BRB0331','HLY0331') and PriceTypeCode = 'F'
)
select L1.PointOfConnectionCode,TraderCode,NetworkParticipantCode, 
Min(L1.TradingDate) as FromDate, Max(L1.TradingDate) as ToDate,
round(sum(MWh),3) as Total_offtake_MWh, round(sum(DollarsPerMegawattHour*MWh),2) as Load_cost
from L1 inner join L2 
on L1.TradingDate = L2.TradingDate 
and L1.TradingPeriodNumber = L2.TradingPeriodNumber 
and L1.PointOfConnectionCode = L2.PointOfConnectionCode
group by all
Order by PointOfConnectionCode

In [0]:
With L1 as (
  select TradingDate, TradingPeriodNumber, TradingPeriodStartTime ,PointOfConnectionCode,TraderCode,NetworkParticipantCode, KilowattHours/1000 as Offtake_MWh 
  from ea_prd.gold.factreconciledinjectionandofftake
  where TradingDate >= '2025-01-01' and FlowDirection = 'Offtake' and KilowattHours > 0
  and (PointOfConnectionCode = 'BRB0331' and TraderCode = 'MERI' and NetworkParticipantCode = 'MERI')
), L2 as (
  select TradingDate, TradingPeriodNumber, TradingPeriodStartTime, PointOfConnectionCode,TraderCode,NetworkParticipantCode, KilowattHours/1000 as Injection_MWh 
  from ea_prd.gold.factreconciledinjectionandofftake
  where TradingDate >= '2025-01-01' and FlowDirection = 'Injection' and KilowattHours > 0
  and (PointOfConnectionCode = 'BRB0331' and TraderCode = 'MERI' and NetworkParticipantCode = 'MERI')
), L3 as (
  select TradingDate, TradingPeriodNumber, TradingPeriodStartTime, PointOfConnectionCode,TraderCode,NetworkParticipantCode from L1
  union 
  select TradingDate, TradingPeriodNumber, TradingPeriodStartTime, PointOfConnectionCode,TraderCode,NetworkParticipantCode from L2
), L4 as (select distinct * from L3)
, L5 as (
  select L4.*, offtake_mwh, injection_mwh from L4 
  left join L1
  on L4.TradingDate = L1.TradingDate and L4.TradingPeriodNumber = L1.TradingPeriodNumber 
  and L4.PointOfConnectionCode = L1.PointOfConnectionCode
  and L4.TraderCode = L1.TraderCode and L4.NetworkParticipantCode = L1.NetworkParticipantCode
  left join L2
  on L4.TradingDate = L2.tradingdate and L4.TradingPeriodNumber = L2.TradingPeriodNumber 
  and L4.PointOfConnectionCode = L2.PointOfConnectionCode
  and L4.TraderCode = L2.TraderCode and L4.NetworkParticipantCode = L2.NetworkParticipantCode
  order by L4.tradingdate, L4.tradingperiodnumber
), L6 as (
  select tradingdate, 
  casesum(offtake_mwh) as offtake_mwh, sum(injection_mwh) as injection_mwh from L5
  group by tradingdate
  order by tradingdate
)
select * from L6



Databricks visualization. Run in Databricks to view.

In [0]:
select TradingDate, TradingPeriodNumber,
ConstraintTypeCode, ConstraintDirectionIndicator, Dollars, Megawatts
from ea_prd.gold.factconstrainedonoff 
where TradingDate between '2024-01-01' and '2024-01-31' 
and PointOfConnectionCode = 'HLY0331' and ParticipantCode  in ('NPEL','OPHL') and Dollars > 0
Order by Dollars desc

In [0]:
Select TraderCode, NetworkParticipantCode,KilowattHours/500 as MW,* from ea_prd.gold.factreconciledinjectionandofftake 
where TradingDate = '2024-01-25' and TradingPeriodNumber = 12 
and FlowDirection = 'Offtake'
and PointOfConnectionCode = 'HLY0331' --and TraderCode = 'NPEL' --and NetworkParticipantCode = 'WAIK'

In [0]:
Select a.TradingDate, a.TradingPeriodNumber, a.IntervalDateTime, a.TrancheNumber,
a.SPDOfferOutputMegawatts as Offer_MW, a.SPDOfferPriceDollarsPerMegawattHour as Offer_Pr, 
a.SPDGenerationMegawatts as Cleared_MW, a.SPDDollarsPerMegawattHour as RTD_Price,
b.DollarsPerMegawattHour as Final_Price, *
from ea_prd.gold.factspdbidsandoffers a 
inner join ea_prd.gold.factpredispatchandfinalenergyprices b
on (a.TradingDate = b.TradingDate and a.TradingPeriodNumber = b.TradingPeriodNumber 
and a.PointOfConnectionCode = b.PointOfConnectionCode
and b.PriceTypeCode = 'F')
where 
-- TradingDate = '2024-01-28' and TradingPeriodNumber = 9 
a.TradingDate = '2024-01-25' and a.TradingPeriodNumber = 12 
and a.PointOfConnectionCode = 'HLY0331'
and CaseTypeCode = 'RTD' and OfferTypeCode = 'ENDL'
Order by IntervalDateTime

In [0]:
select * from ea_prd.gold.factspdbidsandoffers where PointOfConnectionCode = 'HLY0331' and CaseTypeCode = 'RTD'
order by TradingDate

In [0]:
select PointOfConnectionCode || ' ' || UnitCode as PnodeName,TradingDate,TradingPeriodNumber as Period,
IntervalDateTime as `Interval`, ProductType || ' ' || ProductClass as Product, SPDMegawatts 
from datahub.wholesale.spdnodal
where TradingDate >= '2024-01-01' and CaseTypeCode = 'RTD'
and PointOfConnectionCode = 'BRB0331' --and UnitCode = 'RHO99' 
and SPDMegawatts <> 0
order by IntervalDateTime

In [0]:
select PointOfConnectionCode || ' ' || UnitCode as PnodeName , TradingDate, TradingPeriodNumber as Period,
DispatchInstructionSentDateTime, ProductType || ' ' || ProductClass as Product, DispatchedMegawatts, SPDCaseID
from datahub.wholesale.dispatchinstructions a
where TradingDate >= '2024-01-01' and PointOfConnectionCode = 'HLY0331' and ProductType = 'Energy' and  ProductClass = 'Offtake'

In [0]:
select * from ea_prd.gold.factdispatchinstructions 
where TradingDate >= '2024-01-01' and PointOfConnectionCode = 'HLY0331' and UnitCode = 'RHO99'
and DispatchTypeCode = 'DD'

In [0]:
with L1 as (
  select DATE(TRadingDate) as TradingDate, int(TradingPeriod) as TradingPeriodNumber, CaseID, int(InterimTimeWeightSeconds) as NumberofSecs 
  from ea_prd.bronze.casefilepublishtimes where InterimTimeWeightSeconds <> '0' 
), L2 as (
  Select a.CaseID, a.TradingDate, a.TradingPeriodNumber, a.IntervalDateTime, b.NumberofSecs,
  avg(a.SPDClearedFIRMegawatts) as Cleared_FIR_MW, avg(a.SPDClearedFIRMegawatts) as Cleared_SIR_MW
  from ea_prd.gold.factspdbidsandoffers a inner join L1 b
  on (a.TradingDate = b.TradingDate and a.TradingPeriodNumber = b.TradingPeriodNumber and a.CaseID = b.CaseID)
  where a.TradingDate >= '2024-01-01'  and Month(a.TradingDate) = 4
  and a.PointOfConnectionCode = 'HLY0331'
  --and UnitCode in ('RHO0','RHO99')
  and OfferTypeCode not in ('ENOF','ENDL')
  Group By all
), L3 as (
  select TradingDate, TradingPeriodNumber, 
  round(sum(Cleared_FIR_MW*NumberofSecs)/sum(NumberofSecs),3) as Cleared_FIR_MW, 
  round(sum(Cleared_SIR_MW*NumberofSecs)/sum(NumberofSecs),3) as Cleared_SIR_MW
  from L2
  Group by TradingDate, TradingPeriodNumber
), L4 as (
select a.*, b.FIRDollarsPerMegawattHour, b.SIRDollarsPerMegawattHour 
from L3 a inner join ea_prd.gold.factpredispatchandfinalreserveprices b
on (a.TradingDate = b.TradingDate and a.TradingPeriodNumber = b.TradingPeriodNumber 
and b.PointOfConnectionCode = 'HAY2201' and b.PriceTypeCode = 'F')
), L5 as (
select Year(TradingDate) as `Year`, month(TradingDate) as `Month`, 
round(sum(Cleared_FIR_MW * FIRDollarsPerMegawattHour),2) as Revenue_FIR, 
round(sum(Cleared_SIR_MW * SIRDollarsPerMegawattHour),2) as Revenue_SIR 
from L4
Group by all
Order by Year, Month
)
select * from L4

In [0]:
select a.TradingPeriodNumber, DollarsPerMegawattHour, FIRDollarsPerMegawattHour, SIRDollarsPerMegawattHour 
from ea_prd.gold.factpredispatchandfinalreserveprices a
inner join ea_prd.gold.factpredispatchandfinalenergyprices b
on a.TradingDate = b.TradingDate and a.TradingPeriodNumber = b.TradingPeriodNumber 
and a.PointOfConnectionCode = b.PointOfConnectionCode and a.PriceTypeCode = b.PriceTypeCode
where a.TradingDate = '2024-04-20' and a.PriceTypeCode = 'F' and a.PointOfConnectionCode = 'HAY2201'
Order by TradingPeriodNumber

In [0]:
select * from ea_prd.gold.factspdbidsandoffers 
where TradingDate = '2024-04-20' and CaseTypeCode = 'RTD' and PointOfConnectionCode = 'HLY0331' and TradingPeriodNumber = 20
and `Interval` = '20-APR-2024 09:30'
Order by TradingPeriodNumber

In [0]:
select * from ea_prd.gold.factpredispatchandfinalreserveprices
where TradingDate = '2024-04-20' and TradingPeriodNumber = 20 and PointOfConnectionCode = 'HAY2201'--and PriceTypeCode = 'F'
Order by PriceTypeCode, RunDateTime